# Workload of the day

<img src=images/maui-workload.png>

<img src=images/maui-projects.png>

## Xamarin before acquisition

- https://www.mono-project.com
- https://github.com/mono
- https://en.wikipedia.org/wiki/Mono_%28software%29
- https://tirania.org/blog/archive/2011/May-16.html

## Xamarin after acquisition

<img src=images/xamarin-forms.png width=800>

<img src=images/dotnet5_platform.png width=800>
- https://devblogs.microsoft.com/dotnet/introducing-net-5

## .NET MAUI

Multiplatform App UI
- https://dotnet.microsoft.com/en-us/apps/maui
- https://github.com/dotnet/maui

<img src=images/maui-weather.png width=900>

__Resources__

__Github__
- https://github.com/dotnet/maui-samples
- https://github.com/jsuarezruiz/dotnet-maui-showcase
- https://github.com/jsuarezruiz/awesome-dotnet-maui

__Youtube__
- https://www.youtube.com/@dotnet
    - Xamarin for Beginners; https://www.youtube.com/watch?v=JH8ekYJrFHs&list=PLdo4fOcmZ0oU10SXt2W58pu2L0v2dOW-1
    - .NET MAUI for Beginners; https://www.youtube.com/watch?v=Hh279ES_FNQ&list=PLdo4fOcmZ0oUBAdL2NwBpDs32zwGqb9DY



# Threading 101

In [ ]:
// Java
Runnable r = new Runnable() {
    @Override
    public void run() {
        // do something
    }
};

Thread t = new Thread(r);
t.start();
// do something else
t.join();

In [ ]:
// Java
Thread t = new Thread(new Runnable() {
    public void run() {
        // do something
    }
});
t.start();
// do something else
t.join();

In [ ]:
using System.Threading;

var thread = new Thread(new ThreadStart(() =>
{
    // do something
}));

thread.Start();
// do something else
thread.Join();

In [ ]:
using System.Threading;

var thread = new Thread(() =>       // this is ThreadStart with inference
{
    // do something
});

thread.Start();
// do something else
thread.Join();

- https://docs.oracle.com/javase/8/docs/api/java/lang/Runnable.html
- https://learn.microsoft.com/en-us/dotnet/api/system.threading.threadstart
    - ThreadStart is a delegate; even if we have similar looking signature; type of delegates were different
    - Having Action<..> and Func<..> simplfied things; but needed newer APIs

In [ ]:
using System.Threading.Tasks;   // Tasks based; i think 4.5 / Windows 8 time frame

var task1 = Task.Factory.StartNew(() =>
{
    // do something
});
//task1.Wait();

var task2 = Task.Run(() =>
{
    // do something
});
//task2.Wait();

Task.WaitAll(task1, task2);

## Example

In [ ]:
using System.Diagnostics;
using System.IO;
using System.Threading;
using System.Threading.Tasks;

void scanFolder(string folder, bool inParallel)
{
    long totalSize = 0;
    var stopWatch = new Stopwatch();
    stopWatch.Start();

    string[] files = Directory.GetFiles(folder);

    if (!inParallel)
    {
        foreach(var f in files)
        {
            var fi = new FileInfo(f);
            totalSize += fi.Length;
        }
    }
    else
    {
        Parallel.For(0, files.Length, index =>
        {
            var fi = new FileInfo(files[index]);
            long size = fi.Length;
            Interlocked.Add(ref totalSize, size);
        });
    }

    stopWatch.Stop();

    Console.WriteLine($"{files.Length:N0} files, {totalSize:N0} bytes, took {stopWatch.Elapsed}");
}

scanFolder(@"C:\Windows\System32", inParallel: false);
scanFolder(@"C:\Windows\System32", inParallel: true);

__Further Readings__
- https://learn.microsoft.com/en-us/dotnet/standard/threading/managed-threading-basics

## C# Wish: Java Inline Implementation

In [ ]:
// Duck Typing
interface IBird
{
    void MakeSound();
    void Fly();
}

var duck = new  // someway to tell that this shape implemented iBird
                // may be like IBird duck = new { }
{
    MakeSound = new Action(() => { Console.WriteLine("Making sound..."); }),
    Fly = new Action(() => {})
};

duck.MakeSound();

void use(dynamic duck)
{
    duck.themethodthatdoestexist(); //compiler will not help; if we mistype; we can have runtime errors like Python and other dynamic languages
}

# Volatile

- https://learn.microsoft.com/en-us/dotnet/csharp/language-reference/keywords/volatile
    - a field might be modified by multiple threads that are executing at the same time
    - The compiler, the runtime system, and even hardware may rearrange reads and writes to memory locations for performance reasons
- https://ericlippert.com/2011/06/16/atomicity-volatility-and-immutability-are-different-part-three
    - make sure that the compiler and the jitter do not perform any code reordering or register caching optimizations on this variable
    - tell the processors to do whatever it is they need to do to ensure that I am reading the latest value, even if that means halting other processors and making them synchronize main memory with their caches
    - its a lie
    - they provide weaker guarantees about how memory accesses before and after reads and writes may be observed to be ordered with respect to each other

In [ ]:
class VolatileTest
{
    static volatile int sharedStorage;

    public void Test(int i)
    {
        sharedStorage = i;
    }
}

var vt1 = new VolatileTest();
var vt2 = new VolatileTest();
vt1.Test(1);    //what if they are in seperate threads
vt2.Test(5);    //is volatile enough or should we do something more

## Juriba Example

In [ ]:
using System.Collections.Concurrent;

var flags = new ConcurrentDictionary<string, bool>();

bool setFlag(string key) => flags.TryAdd(key, true); // from concurrent dictionary; if we succeed we get true
// indirectly one thread who will be able to add flag will get to know about it

In [ ]:
if (setFlag("Connection_Successful"))
    Console.WriteLine("Connection is successfully established."); // we dont want to log too much; just once

string vmName = "AzureVM11";
string flagKey = $"Machine.InternalDnsName:{vmName}";
if (setFlag(flagKey))
    Console.WriteLine($"Failing to determine DNS name of {vmName}");

# Singleton 1️⃣

<img src=images/singleton-meme.jpg>

We dont need to think about it that much due to Dependency Injection etc; but lets see what it takes

In [ ]:
using System.Threading;

class Singleton
{
    static readonly object lockObject = new object();
    static volatile Singleton instance; //volatile importance

    Singleton() //why it private
    {
    }

    public static Singleton Instance
    {
        get
        {
            if (instance == null)
            {
                lock (lockObject)
                {
                    if (instance == null) // why double null check
                        instance = new Singleton();
                }
            }
            return instance;
        }
    }
}

## Singleton with Database

In [ ]:
#r "nuget: Microsoft.EntityFrameworkCore.SqlServer, 8.0.2"

In [ ]:
using Microsoft.EntityFrameworkCore;

class Reading
{
    public int DataId { get; set; }

    public int Number1 { get; set; }
    public int Number2 { get; set; }
    public int Number3 { get; set; }
    public int Number4 { get; set; }
    public int Number5 { get; set; }

    public double WideNumber1 { get; set; }
    public double WideNumber2 { get; set; }
    public double WideNumber3 { get; set; }
    public double WideNumber4 { get; set; }
    public double WideNumber5 { get; set; }

    public string String1 { get; set; }
    public string String2 { get; set; }
    public string String3 { get; set; }
    public string String4 { get; set; }
    public string String5 { get; set; }
}

class JiraDbContext : DbContext
{
    public DbSet<Reading> Readings { get; set; }

    protected override void OnConfiguring(DbContextOptionsBuilder optionsBuilder)
    {
        var connectionString = "Server=.;Database=JiraDb;Trusted_Connection=True;TrustServerCertificate=True;";
        optionsBuilder.UseSqlServer(connectionString);
    }

    protected override void OnModelCreating(ModelBuilder modelBuilder)
    {
        //base.OnModelCreating(modelBuilder);
        modelBuilder.Entity<Reading>(entity =>
        {
            entity.HasKey(e => e.DataId);
            entity.HasIndex(e => e.DataId).IsUnique();
        });
    }
}

In [ ]:
// this cell can take some time; due to lot of random numbers
// may be we can generate them all before hand using Parallel loop and store them in some stack consume them

var random = new Random(Guid.NewGuid().GetHashCode());

using(var db = new JiraDbContext())
{
    db.Database.EnsureCreated();

    for(int i = 1; i <= 25000; i ++) //lets generate some random data
    {
        db.Readings.Add(new Reading()
        {
            Number1 = random.Next(), Number2 = random.Next(), Number3 = random.Next(),
            Number4 = random.Next(), Number5 = random.Next(),

            WideNumber1 = random.Next() * random.Next(),
            WideNumber2 = random.Next() * random.Next(),
            WideNumber3 = random.Next() * random.Next(),
            WideNumber4 = random.Next() * random.Next(),
            WideNumber5 = random.Next() * random.Next(),

            String1 = random.Next().GetHashCode().ToString(),
            String2 = random.Next().GetHashCode().ToString(),
            String3 = random.Next().GetHashCode().ToString(),
            String4 = random.Next().GetHashCode().ToString(),
            String5 = random.Next().GetHashCode().ToString()
        });
        db.SaveChanges();
    }
}

In [ ]:
using System.Threading;

class Singleton
{
    static readonly object lockObject = new object();
    static volatile Singleton instance;

    Reading[] data = null;

    void print(string s)
    {
        Console.WriteLine($"{DateTime.Now} {s}");
    }

    Singleton() 
    {
        print("ctor started");
        using(var db = new JiraDbContext())
        {
            var q = from d in db.Readings
                    select d;
            data = q.ToArray();
        }
        print("ctor finished");
    }

    public Reading[] Data => data;

    public static Singleton Instance
    {
        get
        {
            if (instance == null)
            {
                lock (lockObject)
                {
                    if (instance == null)
                    {
                        instance = new Singleton();
                    }
                }
            }
            return instance;
        }
    }
}

In [ ]:
using System.Linq;

void firstRandomRow()
{
    var randomRow = Singleton.Instance.Data.OrderBy(d => Guid.NewGuid()).FirstOrDefault();
    if (null != randomRow)
        Console.WriteLine($"{DateTime.Now} [{Thread.CurrentThread.ManagedThreadId}] {randomRow.DataId} is selected");
}

var tasks = new []
{
    new Task(firstRandomRow),
    new Task(firstRandomRow),
    new Task(firstRandomRow),
    new Task(firstRandomRow),
    new Task(firstRandomRow)
};

Parallel.ForEach(tasks, task =>
{
    task.Start();
});

await Task.WhenAll(tasks).ContinueWith(done =>
{
    Console.WriteLine($"{DateTime.Now} Completed...");
});

## Simple Singleton

In [ ]:
class Singleton
{
    static Singleton singleton = new();

    Singleton()
    {
        // your initialization logic
    }

    public static Singleton Instance { get => singleton; }
}

//people on internet say its not thread safe; lets try

In [ ]:
using System.Threading;

class Singleton
{
    static Singleton singleton = new();

    Singleton()
    {
        Console.WriteLine($"ctor started in {Thread.CurrentThread.ManagedThreadId}");
        Thread.Sleep(5000);
        Console.WriteLine($"ctor finished in {Thread.CurrentThread.ManagedThreadId}");
    }

    public static Singleton Instance { get => singleton; }

    public override string ToString() => "Singleton";
}


Console.WriteLine("Setting things up");

var inline = new ThreadStart(() =>
{
    Console.WriteLine($"Action is running in {Thread.CurrentThread.ManagedThreadId}");
    Console.WriteLine(Singleton.Instance);
});

var t1 = new Thread(inline);
var t2 = new Thread(inline);

t1.Start();
t2.Start();

Thread.Sleep(2000);
Console.WriteLine("Joining threads");
t1.Join();
t2.Join();

Console.WriteLine("Done...");

__Further Readings__

For the whole list
- https://refactoring.guru/design-patterns/singleton/csharp/example

Last year we had Design Pattern sessions; material is available @ https://github.com/khurram-uworx/design-patterns